## GROUP 17 -DESIGN AND IMPLEMENTATION OF A SMART FOOD SUPPLY CHAIN MANAGEMENT SYSTEM FOR RESTAURANTS IN MAKERERE UNIVERSITY, KAMPALA

### UNDERSTANDING THE PROBLEM 
The food supply chain ecosystem for restaurants within Makerere University, located in Kampala, is characterized by significant variability in demand, fluctuating supplier reliability, and seasonal consumption shifts tied to the university’s academic calendar. Demand patterns within the university environment are strongly influenced by semester schedules, examination periods, holidays, student population density, and institutional events. These factors introduce uncertainty into procurement planning, inventory management, and pricing decisions for campus-based restaurants.
As a result, restaurants frequently experience stockouts during peak academic periods, overstocking during recess, increased food waste, unstable supplier coordination, and reduced profit margins.

The datasets used for our mnagement system are Makerere Cafeteria.csv and Makerere Kiosk Transactions.csv that we generated to guide us in our analysis.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris, load_wine, load_digits, fetch_openml

# Imports for data loading, preprocessing (scaling) and visualization in a Jupyter notebook



import matplotlib.pyplot as plt

from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
    Normalizer,
    PowerTransformer,
    OneHotEncoder,
    LabelEncoder
)


sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)

#THE DATA STRUCTURE

In [32]:

# Load the dataset
dataset_path = "Makerere Kiosk Transactions.csv"
df = pd.read_csv(dataset_path)

# number of columns
print("Number of columns:", df.shape[1])

# missing values per column
print("\nMissing values per column:")
print(df.isnull().sum())

# datatypes and non-null counts
print("\nDataFrame info:")
df.info()
df.head()
df.shape

Number of columns: 17

Missing values per column:
Transaction_ID        0
Date                  0
Time                  0
Hour                  0
Day_of_Week           0
Academic_Period       0
Is_Weekend            0
Near_Lecture_Start    0
Kiosk_ID              0
Kiosk_Name            0
Kiosk_Location        0
Item                  0
Quantity              0
Unit_Price_UGX        0
Total_Amount_UGX      0
Payment_Method        0
Price_vs_Base_Pct     0
dtype: int64

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119094 entries, 0 to 119093
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Transaction_ID      119094 non-null  object 
 1   Date                119094 non-null  object 
 2   Time                119094 non-null  object 
 3   Hour                119094 non-null  int64  
 4   Day_of_Week         119094 non-null  object 
 5   Academic_Period     119094 non-null  object 
 

(119094, 17)

In [ ]:

# Load the second dataset
dataset_path = "Makerere Cafeteria.csv"
df = pd.read_csv(dataset_path)

# number of columns
print("Number of columns:", df.shape[1])

# missing values per column
print("\nMissing values per column:")
print(df.isnull().sum())

# datatypes and non-null counts
print("\nDataFrame info:")
df.info()
df.describe
df.head()


Number of columns: 28

Missing values per column:
Date                   0
Day_of_Week            0
Academic_Period        0
Is_Weekend             0
Meal                   0
Portions_Prepared      0
Portions_Sold          0
Waste_Portions         0
Waste_Pct              0
Price_UGX              0
Revenue_UGX            0
Ingredient_Cost_UGX    0
Waste_Cost_UGX         0
Gross_Profit_UGX       0
Posho_Flour_kg         0
Beans_kg               0
Cooking_Oil_L          0
Matooke_kg             0
Groundnuts_kg          0
Rice_kg                0
Chicken_kg             0
Offal_kg               0
Onions_kg              0
Irish_Potatoes_kg      0
Eggs_units             0
Wheat_Flour_kg         0
Cabbage_kg             0
Tomatoes_kg            0
dtype: int64

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2196 entries, 0 to 2195
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date    

Index(['Date', 'Day_of_Week', 'Academic_Period', 'Is_Weekend', 'Meal',
       'Portions_Prepared', 'Portions_Sold', 'Waste_Portions', 'Waste_Pct',
       'Price_UGX', 'Revenue_UGX', 'Ingredient_Cost_UGX', 'Waste_Cost_UGX',
       'Gross_Profit_UGX', 'Posho_Flour_kg', 'Beans_kg', 'Cooking_Oil_L',
       'Matooke_kg', 'Groundnuts_kg', 'Rice_kg', 'Chicken_kg', 'Offal_kg',
       'Onions_kg', 'Irish_Potatoes_kg', 'Eggs_units', 'Wheat_Flour_kg',
       'Cabbage_kg', 'Tomatoes_kg'],
      dtype='object')

In [ ]:
# read the two sources into separate dataframes
kiosk = pd.read_csv("Makerere Kiosk Transactions.csv")
cafeteria = pd.read_csv("Makerere Cafeteria.csv")

# strip commas and convert to numeric
def _clean_money(col):
    return pd.to_numeric(col.astype(str).str.replace(",", ""),
                         errors="coerce")

for d in (kiosk, cafeteria):
    for money in ("Unit_Price_UGX", "Total_Amount_UGX"):
        if money in d.columns:
            d[money] = _clean_money(d[money])

# stitch them together
combined = pd.concat([kiosk, cafeteria], ignore_index=True)

# basic campus totals
total_campus_revenue_ugx = combined["Total_Amount_UGX"].sum()
total_portions_served   = combined["Quantity"].sum()
revenue_per_portion_ugx = (
    total_campus_revenue_ugx / total_portions_served
    if total_portions_served else np.nan
)

# look for a cost column to compute margins
cost_columns = [c for c in combined.columns if "cost" in c.lower()]
if cost_columns:
    cost_col = cost_columns[0]
    total_cost = combined[cost_col].sum()
    campus_gross_margin_pct = (total_campus_revenue_ugx - total_cost) / total_campus_revenue_ugx * 100
    aste_cost_pct_of_revenue = total_cost / total_campus_revenue_ugx * 100
else:
    campus_gross_margin_pct = np.nan
    aste_cost_pct_of_revenue = np.nan

# kiosk‑level revenue share (uses only the kiosk dataset)
if "Kiosk_ID" in kiosk.columns:
    kiosk_revenue = kiosk.groupby("Kiosk_ID")["Total_Amount_UGX"].sum()
    kiosk_revenue_share_pct = kiosk_revenue / total_campus_revenue_ugx * 100
else:
    kiosk_revenue_share_pct = pd.Series(dtype=float)

# summary dataframe of the requested columns
summary = pd.DataFrame({
    "total_campus_revenue_ugx": [total_campus_revenue_ugx],
    "campus_gross_margin_pct": [campus_gross_margin_pct],
    "total_portions_served":   [total_portions_served],
    "revenue_per_portion_ugx": [revenue_per_portion_ugx],
    "aste_cost_pct_of_revenue":[aste_cost_pct_of_revenue]
})

print(summary)
print("\nkiosk revenue share (first few kiosks):")
print(kiosk_revenue_share_pct.head())

# `combined` now contains every row from both original files, should you
# need to work with it further.

In [ ]:
caf   = pd.read_csv('Makerere Cafeteria.csv')
kiosk = pd.read_csv('Makerere Kiosk Transactions.csv')
# Currency columns are stored as strings with commas
# Cafeteria currency columns
caf_currency_cols = ['Price_UGX', 'Revenue_UGX', 'Ingredient_Cost_UGX',
                     'Waste_Cost_UGX', 'Gross_Profit_UGX']
for col in caf_currency_cols:
    caf[col] = caf[col].str.replace(',', '').astype(float)

# Kiosk currency columns
kiosk_currency_cols = ['Unit_Price_UGX', 'Total_Amount_UGX']
for col in kiosk_currency_cols:
    kiosk[col] = kiosk[col].str.replace(',', '').astype(float)

# Parse dates
caf['Date']   = pd.to_datetime(caf['Date'])
kiosk['Date'] = pd.to_datetime(kiosk['Date'])

print("Data types fixed successfully.")
print(caf[caf_currency_cols].head(2))

Data types fixed successfully.
   Price_UGX  Revenue_UGX  Ingredient_Cost_UGX  Waste_Cost_UGX  \
0     3000.0     492000.0             229500.0          8100.0   
1     4000.0     524000.0             292000.0         30000.0   

   Gross_Profit_UGX  
0          262500.0  
1          232000.0  


In [38]:
# Aggregate cafeteria data to one row per day
caf_daily = caf.groupby('Date').agg(
    caf_total_portions_prepared  = ('Portions_Prepared',   'sum'),
    caf_total_portions_sold      = ('Portions_Sold',       'sum'),
    caf_total_waste_portions     = ('Waste_Portions',      'sum'),
    caf_avg_waste_pct            = ('Waste_Pct',           'mean'),
    caf_total_revenue_ugx        = ('Revenue_UGX',         'sum'),
    caf_total_ingredient_cost_ugx= ('Ingredient_Cost_UGX', 'sum'),
    caf_total_waste_cost_ugx     = ('Waste_Cost_UGX',      'sum'),
    caf_total_gross_profit_ugx   = ('Gross_Profit_UGX',    'sum'),
    # Daily ingredient totals
    caf_posho_flour_kg           = ('Posho_Flour_kg',      'sum'),
    caf_beans_kg                 = ('Beans_kg',            'sum'),
    caf_cooking_oil_l            = ('Cooking_Oil_L',       'sum'),
    caf_matooke_kg               = ('Matooke_kg',          'sum'),
    caf_groundnuts_kg            = ('Groundnuts_kg',       'sum'),
    caf_rice_kg                  = ('Rice_kg',             'sum'),
    caf_chicken_kg               = ('Chicken_kg',          'sum'),
    caf_offal_kg                 = ('Offal_kg',            'sum'),
    caf_onions_kg                = ('Onions_kg',           'sum'),
    caf_irish_potatoes_kg        = ('Irish_Potatoes_kg',   'sum'),
    caf_eggs_units               = ('Eggs_units',          'sum'),
    caf_wheat_flour_kg           = ('Wheat_Flour_kg',      'sum'),
    caf_cabbage_kg               = ('Cabbage_kg',          'sum'),
    caf_tomatoes_kg              = ('Tomatoes_kg',         'sum'),
    caf_meals_offered            = ('Meal',                'count'),
).reset_index()

print("Cafeteria daily shape:", caf_daily.shape)
caf_daily.head(3)

Cafeteria daily shape: (366, 24)


,Date,caf_total_portions_prepared,caf_total_portions_sold,caf_total_waste_portions,caf_avg_waste_pct,caf_total_revenue_ugx,caf_total_ingredient_cost_ugx,caf_total_waste_cost_ugx,caf_total_gross_profit_ugx,caf_posho_flour_kg,...,caf_rice_kg,caf_chicken_kg,caf_offal_kg,caf_onions_kg,caf_irish_potatoes_kg,caf_eggs_units,caf_wheat_flour_kg,caf_cabbage_kg,caf_tomatoes_kg,caf_meals_offered
0,2024-02-01,782,724,58,8.150000,2641500.0,1359640.0,130690.0,1281860.0,42.5,...,20.4,18.4,12.3,4.1,35.7,333,21.6,9.0,7.2,6
1,2024-02-02,681,566,115,16.933333,2087500.0,1171440.0,194220.0,916060.0,39.5,...,17.8,16.0,11.2,3.8,30.1,291,19.4,8.1,6.5,6
2,2024-02-03,243,216,27,12.300000,779000.0,415240.0,54270.0,363760.0,13.0,...,6.6,5.9,4.0,1.4,9.4,105,7.8,3.2,2.6,6


In [39]:
#  Aggregate kiosk data to one row per day

kiosk_daily = kiosk.groupby('Date').agg(
    kiosk_total_transactions   = ('Transaction_ID',    'count'),
    kiosk_total_revenue_ugx    = ('Total_Amount_UGX',  'sum'),
    kiosk_avg_transaction_ugx  = ('Total_Amount_UGX',  'mean'),
    kiosk_items_sold           = ('Quantity',          'sum'),
    kiosk_cash_pct             = ('Payment_Method',    lambda x: round((x == 'Cash').mean() * 100, 1)),
    kiosk_momo_pct             = ('Payment_Method',    lambda x: round((x == 'MTN MoMo').mean() * 100, 1)),
    kiosk_airtel_pct           = ('Payment_Method',    lambda x: round((x == 'Airtel Money').mean() * 100, 1)),
    kiosk_discounted_txns      = ('Price_vs_Base_Pct', lambda x: (x < 0).sum()),
    kiosk_peak_hour            = ('Hour',              lambda x: x.value_counts().idxmax()),
    kiosk_near_lecture_sales   = ('Near_Lecture_Start',lambda x: x.sum()),
).reset_index()

print("Kiosk daily shape:", kiosk_daily.shape)
kiosk_daily.head(3)


Kiosk daily shape: (366, 11)


,Date,kiosk_total_transactions,kiosk_total_revenue_ugx,kiosk_avg_transaction_ugx,kiosk_items_sold,kiosk_cash_pct,kiosk_momo_pct,kiosk_airtel_pct,kiosk_discounted_txns,kiosk_peak_hour,kiosk_near_lecture_sales
0,2024-02-01,162,205000.0,1265.432099,162,52.5,33.3,14.2,17,12,21
1,2024-02-02,161,177750.0,1104.037267,161,60.9,23.0,16.1,18,14,26
2,2024-02-03,37,47250.0,1277.027027,37,64.9,21.6,13.5,6,7,0


In [ ]:
# — Kiosk item pivot (daily sales per item)

item_pivot = (
    kiosk.groupby(['Date', 'Item'])['Quantity']
    .sum()
    .unstack(fill_value=0)
)
# Clean column names
item_pivot.columns = [
    'kiosk_sold_' + c.replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_').lower()
    for c in item_pivot.columns
]
item_pivot = item_pivot.reset_index()

print("Item pivot shape:", item_pivot.shape)
print("Item columns:", item_pivot.columns.tolist())

Item pivot shape: (366, 10)
Item columns: ['Date', 'kiosk_sold_biscuits_packet', 'kiosk_sold_chapati', 'kiosk_sold_groundnuts_packet', 'kiosk_sold_juice_splash', 'kiosk_sold_mandazi_2_pcs', 'kiosk_sold_rolex', 'kiosk_sold_samosa_2_pcs', 'kiosk_sold_soda', 'kiosk_sold_water_bottle']


In [ ]:
#  Kiosk location revenue pivot (daily revenue per kiosk)

loc_pivot = (
    kiosk.groupby(['Date', 'Kiosk_Name'])['Total_Amount_UGX']
    .sum()
    .unstack(fill_value=0)
)
loc_pivot.columns = [
    'kiosk_rev_' + c.replace(' ', '_').replace('(', '').replace(')', '').lower()
    for c in loc_pivot.columns
]
loc_pivot = loc_pivot.reset_index()

print("Location pivot shape:", loc_pivot.shape)
print("Location columns:", loc_pivot.columns.tolist())


Location pivot shape: (366, 8)
Location columns: ['Date', 'kiosk_rev_africa_hall_kiosk', 'kiosk_rev_education_block_kiosk', 'kiosk_rev_engineering_block_kiosk', 'kiosk_rev_freedom_square_kiosk', 'kiosk_rev_main_gate_kiosk', 'kiosk_rev_mitchell_hall_kiosk', 'kiosk_rev_senate_building_kiosk']


In [ ]:
#  Merge all four tables on Date

combined = (
    caf_daily
    .merge(kiosk_daily,  on='Date', how='inner')
    .merge(item_pivot,   on='Date', how='left')
    .merge(loc_pivot,    on='Date', how='left')
)

# Bring back context columns (Day_of_Week, Academic_Period, Is_Weekend)
context = caf[['Date', 'Day_of_Week', 'Academic_Period', 'Is_Weekend']].drop_duplicates('Date')
combined = combined.merge(context, on='Date', how='left')

print("Combined shape after merge:", combined.shape)

Combined shape after merge: (366, 53)


In [ ]:
#  Rolex demand (sold in BOTH cafeteria and kiosk)

rolex_caf = (
    caf[caf['Meal'] == 'Rolex']
    .groupby('Date')['Portions_Sold']
    .sum()
    .reset_index()
    .rename(columns={'Portions_Sold': 'caf_rolex_portions'})
)

rolex_kiosk = (
    kiosk[kiosk['Item'] == 'Rolex']
    .groupby('Date')['Quantity']
    .sum()
    .reset_index()
    .rename(columns={'Quantity': 'kiosk_rolex_units'})
)

combined = combined.merge(rolex_caf,   on='Date', how='left')
combined = combined.merge(rolex_kiosk, on='Date', how='left')

combined['caf_rolex_portions']  = combined['caf_rolex_portions'].fillna(0)
combined['kiosk_rolex_units']   = combined['kiosk_rolex_units'].fillna(0)

print("Rolex columns added.")

Rolex columns added.


In [ ]:
#  DERIVED SUPPLY CHAIN KPIs
# These are the new columns only possible after combining

# 1. Total campus food revenue (cafeteria + all kiosks)
combined['total_campus_revenue_ugx'] = (
    combined['caf_total_revenue_ugx'] + combined['kiosk_total_revenue_ugx']
)

# 2. Campus gross margin % (profit vs total revenue)
combined['campus_gross_margin_pct'] = (
    combined['caf_total_gross_profit_ugx'] / combined['total_campus_revenue_ugx'] * 100
).round(2)

# 3. Total portions served across both outlets
combined['total_portions_served'] = (
    combined['caf_total_portions_sold'] + combined['kiosk_items_sold']
)

# 4. Revenue per portion (average spend per person per day)
combined['revenue_per_portion_ugx'] = (
    combined['total_campus_revenue_ugx'] / combined['total_portions_served']
).round(0)

# 5. Waste cost as % of total revenue
combined['waste_cost_pct_of_revenue'] = (
    combined['caf_total_waste_cost_ugx'] / combined['total_campus_revenue_ugx'] * 100
).round(2)

# 6. Ingredient cost ratio (procurement cost as % of cafeteria revenue)
combined['ingredient_cost_ratio'] = (
    combined['caf_total_ingredient_cost_ugx'] / combined['caf_total_revenue_ugx'] * 100
).round(2)

# 7. Kiosk revenue share (% of campus food revenue from kiosks)
combined['kiosk_revenue_share_pct'] = (
    combined['kiosk_total_revenue_ugx'] / combined['total_campus_revenue_ugx'] * 100
).round(2)

# 8. Demand fulfilment rate (% of prepared portions actually sold)
combined['demand_fulfilment_rate_pct'] = (
    combined['caf_total_portions_sold'] / combined['caf_total_portions_prepared'] * 100
).round(2)

# 9. Procurement efficiency (% of prepared food NOT wasted)
combined['procurement_efficiency'] = (
    (combined['caf_total_portions_prepared'] - combined['caf_total_waste_portions']) /
    combined['caf_total_portions_prepared'] * 100
).round(2)

# 10. Total Rolex demand (cafeteria + kiosk combined)
combined['total_rolex_demand'] = (
    combined['caf_rolex_portions'] + combined['kiosk_rolex_units']
)

# 11. Eggs needed for total Rolex demand (~1 egg per Rolex)
combined['rolex_egg_demand_units'] = combined['total_rolex_demand'].astype(int)

# 12. Wheat flour needed for total Rolex demand (~50g per Rolex)
combined['rolex_wheat_flour_demand_kg'] = (
    combined['total_rolex_demand'] * 0.05
).round(3)

# 13. Demand vs period average (how far above/below normal for that academic period)
period_avg = combined.groupby('Academic_Period')['total_portions_served'].transform('mean')
combined['demand_vs_avg_pct'] = (
    (combined['total_portions_served'] - period_avg) / period_avg * 100
).round(1)

print("All 13 supply chain KPI columns created successfully!")



All 13 supply chain KPI columns created successfully!


In [ ]:
#  Preview the final combined dataset

kpi_cols = [
    'Date', 'Day_of_Week', 'Academic_Period', 'Is_Weekend',
    'total_campus_revenue_ugx',
    'campus_gross_margin_pct',
    'total_portions_served',
    'revenue_per_portion_ugx',
    'waste_cost_pct_of_revenue',
    'ingredient_cost_ratio',
    'kiosk_revenue_share_pct',
    'demand_fulfilment_rate_pct',
    'procurement_efficiency',
    'total_rolex_demand',
    'rolex_egg_demand_units',
    'rolex_wheat_flour_demand_kg',
    'demand_vs_avg_pct',
]
print(f"\nFinal combined dataset: {combined.shape[0]} rows x {combined.shape[1]} columns")
print(f"\nKey Supply Chain KPIs preview:")
combined[kpi_cols].head(10)



Final combined dataset: 366 rows x 68 columns

Key Supply Chain KPIs preview:


,Date,Day_of_Week,Academic_Period,Is_Weekend,total_campus_revenue_ugx,campus_gross_margin_pct,total_portions_served,revenue_per_portion_ugx,waste_cost_pct_of_revenue,ingredient_cost_ratio,kiosk_revenue_share_pct,demand_fulfilment_rate_pct,procurement_efficiency,total_rolex_demand,rolex_egg_demand_units,rolex_wheat_flour_demand_kg,demand_vs_avg_pct
0,2024-02-01,Thursday,Other,False,2846500.0,45.03,886,3213.0,4.59,51.47,7.20,92.58,92.58,208.0,208,10.40,68.4
1,2024-02-02,Friday,Other,False,2265250.0,40.44,727,3116.0,8.57,56.12,7.85,83.11,83.11,155.0,155,7.75,38.2
2,2024-02-03,Saturday,Other,True,826250.0,44.03,253,3266.0,6.57,53.30,5.72,88.89,88.89,68.0,68,3.40,-51.9
3,2024-02-04,Sunday,Other,True,780000.0,43.48,238,3277.0,6.93,54.11,5.26,86.21,86.21,56.0,56,2.80,-54.8
4,2024-02-05,Monday,Sem1_Teaching,False,6111500.0,41.86,1991,3070.0,4.76,52.92,11.09,88.14,88.14,408.0,408,20.40,26.7
5,2024-02-06,Tuesday,Sem1_Teaching,False,5861250.0,41.23,1938,3024.0,5.88,53.46,11.43,86.60,86.60,453.0,453,22.65,23.3
6,2024-02-07,Wednesday,Sem1_Teaching,False,5842750.0,40.21,1923,3038.0,6.88,54.67,11.30,86.03,86.03,419.0,419,20.95,22.4
7,2024-02-08,Thursday,Sem1_Teaching,False,6148750.0,41.49,2019,3045.0,5.55,53.16,11.42,88.03,88.03,477.0,477,23.85,28.5
8,2024-02-09,Friday,Sem1_Teaching,False,4968250.0,39.93,1726,2878.0,6.17,53.68,13.79,87.14,87.14,416.0,416,20.80,9.8
9,2024-02-10,Saturday,Sem1_Teaching,True,1762750.0,40.88,556,3170.0,6.74,54.47,10.23,85.51,85.51,143.0,143,7.15,-64.6


In [ ]:
# Quick summary statistics on KPIs

print("=" * 55)
print("SUPPLY CHAIN KPI SUMMARY — MAKERERE CAMPUS")
print("=" * 55)
print(f"Date range          : {combined['Date'].min().date()} to {combined['Date'].max().date()}")
print(f"Total days          : {combined['Date'].nunique()}")
print(f"\nRevenue")
print(f"  Avg daily campus revenue : UGX {combined['total_campus_revenue_ugx'].mean():>12,.0f}")
print(f"  Avg campus gross margin  : {combined['campus_gross_margin_pct'].mean():.1f}%")
print(f"  Avg kiosk revenue share  : {combined['kiosk_revenue_share_pct'].mean():.1f}%")
print(f"\nDemand")
print(f"  Avg portions served/day  : {combined['total_portions_served'].mean():.0f}")
print(f"  Avg revenue per portion  : UGX {combined['revenue_per_portion_ugx'].mean():,.0f}")
print(f"  Avg fulfilment rate      : {combined['demand_fulfilment_rate_pct'].mean():.1f}%")
print(f"\nWaste & Procurement")
print(f"  Avg waste % of revenue   : {combined['waste_cost_pct_of_revenue'].mean():.1f}%")
print(f"  Avg ingredient cost ratio: {combined['ingredient_cost_ratio'].mean():.1f}%")
print(f"  Avg procurement efficiency: {combined['procurement_efficiency'].mean():.1f}%")
print(f"\nRolex Procurement Signal")
print(f"  Avg daily Rolex demand   : {combined['total_rolex_demand'].mean():.0f} units")
print(f"  Avg eggs needed/day      : {combined['rolex_egg_demand_units'].mean():.0f} units")
print(f"  Avg wheat flour needed   : {combined['rolex_wheat_flour_demand_kg'].mean():.1f} kg")


In [ ]:
#  Save the combined dataset to CSV

combined.to_csv('makerere_supply_chain_combined.csv', index=False)
print("Combined dataset saved as: makerere_supply_chain_combined.csv")
print(f"Total columns: {combined.shape[1]}")
print(f"Total rows   : {combined.shape[0]}")


Combined dataset saved as: makerere_supply_chain_combined.csv
Total columns: 68
Total rows   : 366


##WHY WERE THE FOLLOWING COLUMNS CREATED IN THE NEW DATASET
Here is a thorough explanation of every new column, grounded in the actual numbers from your data.

---

## Why Each New Column Matters

### 1. `total_campus_revenue_ugx`
**What it is:** Cafeteria revenue + all 7 kiosk revenues added together for each day.

**Why it matters:** Neither dataset alone tells you the true size of the Makerere food economy. The cafeteria sees the big meal revenue; the kiosks see the snack and drink revenue. Separately they are partial pictures. Combined, you see the full picture , on a peak semester teaching weekday this reaches **UGX 7.47 million**, while on a Christmas break day it drops to **UGX 215,600**. That is a 35× difference, which has enormous implications for how much cash flow the operation needs to sustain supplier payments during quiet periods.

---

### 2. `campus_gross_margin_pct`
**What it is:** The percentage of total campus revenue that becomes gross profit after ingredient costs.

**Why it matters:** This is the single most important profitability indicator for the operation. Our data shows the margin consistently sits between **33.8% and 47.3%**, averaging **41.4%**. A procurement manager can use this to immediately spot days where ingredient costs spiralled ,a drop below 38% is a red flag that either procurement was wasteful or a supplier charged above-market prices. You cannot calculate this cross-outlet metric from either dataset alone.

---

### 3. `total_portions_served`
**What it is:** Cafeteria portions sold + kiosk items sold, giving total people fed across campus each day.

**Why it matters:** This is the demand signal that drives everything upstream — how much to order, how much to cook, how many staff to deploy. The range in your data is striking: **63 portions on 22 Dec 2024** (Christmas break) versus **2,451 portions** on a peak teaching day. Without combining both outlets, you are underestimating how many people the campus is actually feeding, which leads to systematic under-procurement for the kiosks.

---

### 4. `revenue_per_portion_ugx`
**What it is:** Total campus revenue divided by total portions served — the average spend per person per day.

**Why it matters:** This ranges from **UGX 2,514 to UGX 3,488**, averaging **UGX 3,077**. When this number drops, it means people are gravitating toward cheaper items — which signals pressure on the ingredient mix and purchasing decisions. When it rises during exam periods, it suggests students are buying more kiosk snacks alongside cafeteria meals. Tracking this over time helps you understand whether pricing changes are working.

---

### 5. `waste_cost_pct_of_revenue`
**What it is:** Daily waste cost from the cafeteria expressed as a percentage of total campus revenue.

**Why it matters:** Your campus is losing between **2.45% and 12.36%** of revenue to food waste every single day, averaging **6.3%**. Across the full year that amounts to **UGX 68.7 million in wasted food** — on a total annual revenue of UGX 1.185 billion. Expressed as a percentage of revenue rather than an absolute figure, this becomes something a university administrator can immediately understand and benchmark against. It also reveals that waste is proportionally worse on low-demand days (breaks, weekends), confirming that over-preparation is the root cause rather than poor storage.

---

### 6. `ingredient_cost_ratio`
**What it is:** Total ingredient cost as a percentage of cafeteria revenue only (since kiosks sell packaged items, not freshly cooked).

**Why it matters:** This hovers tightly between **49.5% and 59.8%**, averaging **53.8%**. In food service, the industry benchmark is typically 28–35% for a well-run operation, so at 53.8% Makerere is spending significantly more on ingredients relative to revenue. This column makes that inefficiency visible in one number. It fluctuates daily, so you can correlate spikes with specific meals, suppliers, or market price increases — which is exactly what a supply chain system needs to trigger procurement reviews.

---

### 7. `kiosk_revenue_share_pct`
**What it is:** The percentage of total campus food revenue that the 7 kiosks collectively contribute.

**Why it matters:** Kiosks contribute just **2.3% to 24.9%** of daily revenue, averaging **10.4%**. Crucially, their share is higher on weekends (**8.7%**) relative to total revenue, even though absolute revenue is lower — meaning the kiosks are proportionally more important when the cafeteria is quiet. This tells procurement that kiosk stock cannot be neglected during breaks and weekends simply because cafeteria demand is low. It also benchmarks the kiosks' contribution and could justify investment in expanding kiosk capacity.

---

### 8. `demand_fulfilment_rate_pct`
**What it is:** The percentage of prepared cafeteria portions that were actually sold.

**Why it matters:** Your cafeteria achieves a fulfilment rate of **78.9% to 93.6%**, averaging **87.2%**. Only one day in 366 fell below 80% — 22 December 2024, the quietest day of the year with just 63 portions served. An 87% rate means on average 13% of everything cooked goes unsold. For procurement, this is the gap between what was purchased from suppliers and what was actually needed — a persistent 13% over-procurement that directly becomes the waste cost calculated in column 5.

---

### 9. `procurement_efficiency`
**What it is:** The percentage of prepared portions that were NOT wasted — essentially the inverse of the over-preparation rate.

**Why it matters:** At **87.2% average efficiency**, the kitchen is converting 87 out of every 100 prepared portions into sold food. The goal for a well-run supply chain is 95%+. Tracking this daily allows a supply chain manager to set improvement targets, measure progress, and reward kitchen teams for reducing waste. The fact that it matches `demand_fulfilment_rate_pct` exactly confirms that every unsold portion becomes waste — there is no redistribution or repurposing happening currently.

---

### 10. `total_rolex_demand`
**What it is:** Rolex portions sold in the cafeteria plus Rolex units sold across all 7 kiosks, combined into one daily figure.

**Why it matters:** Rolex is the only item sold in both outlets, making it uniquely important as a procurement signal. It swings dramatically — from just **13 units** on 22 December to **626 units** on 1 October 2024. Without combining both datasets you would only see half the Rolex demand and therefore systematically under-order the ingredients it requires. This single number drives the two procurement columns below.

---

### 11. `rolex_egg_demand_units`
**What it is:** The number of eggs needed each day to fulfil total Rolex demand across campus (1 egg per Rolex).

**Why it matters:** Eggs are perishable, high-turnover, and price-volatile in the Ugandan market. On your busiest day the campus needed **626 eggs just for Rolex**. Multiply that across all egg-based meals (Chips & Eggs, Rolex in both outlets) and you have a precise daily egg procurement target rather than a rough estimate. This column gives the supplier a concrete number to deliver against and enables the cafeteria to negotiate volume-based egg pricing with confidence.

---

### 12. `rolex_wheat_flour_demand_kg`
**What it is:** Kilograms of wheat flour needed each day for total Rolex demand (approximately 50g per Rolex).

**Why it matters:** Wheat flour is a bulk dry good that can be ordered weekly rather than daily, but the order size must reflect peak demand. Your data shows the range is **0.65 kg to 31.3 kg per day**. A weekly procurement plan that ignores kiosk Rolex sales would consistently under-order flour during peak teaching weeks, leading to mid-week stockouts. This column enables accurate weekly flour orders that account for both outlets.

---

### 13. `demand_vs_avg_pct`
**What it is:** How far above or below the average for that academic period each day's demand falls, expressed as a percentage.

**Why it matters:** This is the early warning system for abnormal demand. While the academic period tells you the broad season (teaching, exams, break), `demand_vs_avg_pct` tells you when something unusual is happening within that season. A value of **+68.4%** means demand was 68% above the typical teaching-period day — possibly an event, a graduation, or the first week of term. A value of **-68.1%** means demand collapsed well below even break-period norms. Both are procurement risk signals: one risks a stockout, the other risks excessive waste. This column lets the supply chain system flag those days automatically and prompt a review of that day's order quantities.